In [1]:
import os 
import subprocess 
import numpy 
import torch 
from torch import nn 
from d2l import torch as d2l

通过后端异步处理

In [2]:
def try_xpu(i=0):
    """如果存在,则返回gpu(i),否则返回cpu()"""
    if torch.xpu.device_count() >= i+1:
        return torch.device(f'xpu:{i}')
    return torch.device('cpu')

In [19]:
# GPU计算热身
device = try_xpu()
a = torch.randn(size=(1000, 1000), device=device)
b = torch.mm(a, a)

with d2l.Benchmark('numpy'):
    for _ in range(10):
        a = numpy.random.normal(size=(1000, 1000))
        b = numpy.dot(a, a)

with d2l.Benchmark('torch'):
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)

numpy: 2.2048 sec
torch: 0.0046 sec


In [23]:
with d2l.Benchmark():
    for _ in range(10):
        a = torch.randn(size=(1000, 1000), device=device)
        b = torch.mm(a, a)
    torch.xpu.synchronize(device) # 强制等待所有在指定设备(如GPU/XPU)上的异步操作完成，确保计时的准确性

Done: 0.0322 sec


In [24]:
x = torch.ones((1, 2), device=device)
y = torch.ones((1, 2), device=device)
z = x * y + 2
z

tensor([[3., 3.]], device='xpu:0')